# Extraction Mover

**This tutorial shows how to copy a simple extraction from one team/ data job to another one, independent of the cluster.**



**To do so we first need to connect to the source data model.**

In [ ]:
import pycelonis
from pycelonis import get_celonis

c_source = get_celonis("URL to the team from which you want to copy.", "Specify a valid API key for your source cloud team.")
# All IDs required can be found within the URLs when displaying the related objects in the EMS.
source_data_pool = c_source.pools.find('Name or ID of the source data pool.')
source_data_job = source_data_pool.data_jobs.find("Name or ID of the source data job.")
source_extraction = source_data_job.extractions.ids['ID of the source extraction task.']

In [ ]:
# Print source variables
print(source_data_pool)
print(source_data_job)
print(source_extraction)

**In the next step we connect to the target data pool.**

In [ ]:
c_target = get_celonis("URL to cloud team where you want to copy to.", "Specify a valid API key for your target cloud team.")
target_data_pool = c_target.pools.find("Name or ID of the target data pool.")
target_data_job = target_data_pool.data_jobs.find("Name or ID of the target data job.")

In [ ]:
# Print target variables
print(target_data_pool)
print(target_data_job)

**Create target extraction.**

In [ ]:
# Print source extraction name and type
print(source_extraction.name)

In [ ]:
target_extraction = target_data_job.create_extraction(source_extraction.name)
print(target_extraction.name)

**In this step we save all source global parameter IDs in a dictionary and overwrite them with the target global parameter ID if the parameter exists already in the target data pool.**

In [ ]:
global_vars = {}
for source_var in source_data_pool.variables:                                                          # loop through global parameters of source data pool
    global_vars.update({source_var.id: None})                                                          # save ID of source global parameter
    for target_var in target_data_pool.variables:                                                      # loop through the global parameters of target data pool
        if source_var.data['placeholder'].upper() == target_var.data['placeholder'].upper():           # if the placeholder of a source global parameter and a target global parameter match
            global_vars.update({source_var.id: target_var.id})                                         # match the saved ID of source global parameter wih the target global parameter ID
print("Global parameter configurations saved.")

**This section serves to create the extraction parameters.**

In [ ]:
for source_ex_para in source_extraction.variables:                                                                              # loop through the source extraction parameters
    if source_ex_para['defaultSettings'] is not None:                                                                           # create the connection for the default value to the target global parameters
        if source_ex_para['defaultSettings']['poolVariableId'] is not None:
            target_id = global_vars.get(source_ex_para['defaultSettings']['poolVariableId'])              
            if target_id is None:
                para = target_data_pool.create_pool_parameter(source_data_pool.variables.find(source_ex_para['defaultSettings']['poolVariableId'])) # create the target global parameter if it does not exist yet
                print("Pool Parameter '" + para.name + "' created.")
                target_id = para.id
                global_vars[source_ex_para['defaultSettings']['poolVariableId']] = target_id
            source_ex_para['defaultSettings']['poolVariableId'] = target_id
    if source_ex_para['settings'] is not None:                                                                                   # create the connection for the value to the target global parameters
        if source_ex_para['settings']['poolVariableId'] is not None:
            target_id = global_vars.get(source_ex_para['settings']['poolVariableId'])
            if target_id is None:
                para = target_data_pool.create_pool_parameter(source_data_pool.variables.find(source_ex_para['settings']['poolVariableId']))        # create the target global parameter if it does not exist yet
                print("Pool Parameter '" + para.name + "' created.")
                target_id = para.id
                global_vars[source_ex_para['settings']['poolVariableId']] = target_id
            source_ex_para['settings']['poolVariableId'] = target_id
         
    target_extraction.create_extraction_parameter(source_ex_para)                                                                # create the target transformation parameter
    print("Parameter '" + source_ex_para['name'] + "' created.")

**Add tables in target extraction and overwrite parameter connections for time filters.**

In [ ]:
for table in source_extraction.tables.data:
    
    if table.data['creationDateParameterStart'] is not None or table.data['creationDateParameterEnd'] is not None:
        #None or table.data['changeDateOffsetParameter'] is not None:
        
        temp = table.data.copy()
        
        for para in source_extraction.variables:
            if para['id'] == table.data['creationDateParameterStart']:
                placeholder_start = para['placeholder']
#            if para['id'] == table.data['creationDateParameterEnd']:
#                placeholder_end = para['placeholder']
#            if para['id'] == table.data['changeDateOffsetParameter']:
#                placeholder_change = para['placeholder']
                
        for para in target_extraction.variables:
            if para['placeholder'] == placeholder_start:
                temp['creationDateParameterStart'] = para['id']
 #           if para['placeholder'] == placeholder_end:
 #               temp['creationDateParameterEnd'] = para['id']
 #           if para['placeholder'] == placeholder_change:
 #               temp['changeDateOffsetParameter'] = para['id']
        
        table=temp          
              
    try:
        target_extraction.add_table(table=table)
    except:
        print("Filters for table: '" + table.name + "' are neglected as the table is not found in the indicated source connection.")

print("Congrats you copied the extraction "+ target_extraction.name + "!")